# 4. 최단 거리 버스 정류장의 이름과 X Y 좌표 확인

In [ ]:
import pandas as pd
from pandas import DataFrame
import numbers
import math
from openpyxl import load_workbook
import numpy as np

In [ ]:
senior_df=pd.read_csv('final_senior_center.csv', encoding='utf-8') # 경로당 데이터
#print(senior_df.head(5))

jeonju_df=senior_df["borough"].str.contains("전주시") # 전주시만 추출

df=senior_df[jeonju_df]
print(df["address"].tail(2))

In [ ]:
bus_df=pd.read_csv('busStop.csv', encoding='utf-8') # 버스정류장 데이터
bus_df.head(2)

In [ ]:
class GeoUtil: # x좌표 y좌표 km로 변환해서 x1 y1 x2 y2 차이 구하기
    """
    Geographical Utils
    """
    @staticmethod
    def degree2radius(degree):
        return degree * (math.pi / 180)

    @staticmethod
    def get_harversion_distance(x1, y1, x2, y2, round_decimal_digits=5):
        """
        경위도 (x1,y1)과 (x2,y2) 점의 거리를 반환
        Harversion Formula 이용하여 2개의 경위도간 거래를 구함(단위:Km)
        """
        if x1 is None or y1 is None or x2 is None or y2 is None:
            return None

        R = 6371  # 지구의 반경(단위: km)
        dLon = GeoUtil.degree2radius(x2 - x1)
        dLat = GeoUtil.degree2radius(y2 - y1)

        a = math.sin(dLat / 2) * math.sin(dLat / 2) \
            + (math.cos(GeoUtil.degree2radius(y1)) \
               * math.cos(GeoUtil.degree2radius(y2)) \
               * math.sin(dLon / 2) * math.sin(dLon / 2))
        b = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        return round(R * b, round_decimal_digits)

In [ ]:
center_x=df['X'] # 경로당(마을회관) x 좌표
center_y=df['Y'] # 경로당(마을회관) y 좌표

busStop_x=bus_df['X'] # 버스정류장 x 좌표
busStop_y=bus_df['Y'] # 버스정류장 y 좌표

In [ ]:
min_distance=10000 # 최솟값 초기화
distance=[] # 경로당에서 버스정류장까지 거리 리스트
busStop_name=[] # 최단 거리 버스 정류장 이름 리스트
bus_X=[]; bus_Y=[]

#str(name)

for k in range(len(df)): # 리스트 초기화
    distance.append(0)
    busStop_name.append(0)
    bus_X.append(0)
    bus_Y.append(0)

stop_name=bus_df['busStop']
busX=bus_df['X']; busY=bus_df['Y']

    
for i in range(0, len(df)): # 경로당에서 제일 가까운 버스정류장까지 거리
    for j in range(0, len(bus_df)):
        if (GeoUtil.get_harversion_distance(center_x[i], center_y[i], busStop_x[j], busStop_y[j])) <= min_distance:
            min_distance = GeoUtil.get_harversion_distance(center_x[i], center_y[i], busStop_x[j], busStop_y[j])
            name=stop_name[j]
            X=busX[j]; Y=busY[j]
    distance[i]=min_distance
    busStop_name[i]=name
    bus_X[i]=X; bus_Y[i]=Y
    
    min_distance=10000



In [ ]:
#print(busStop_name)
#print(len(busStop_name))

busStop_df=pd.DataFrame(busStop_name)
#print(busStop_df.tail(2))
busStop_df.columns=["busStop"]
print(busStop_df.head(2))

busX=pd.DataFrame(bus_X); busX.columns=["bus_X"]
busY=pd.DataFrame(bus_Y); busY.columns=["bus_Y"]

In [ ]:
distance_df=pd.DataFrame(distance) # 리스트 -> 데이터프레임
distance_df.columns=['distance'] # column name 변환
print(distance_df.head(10))
print(len(distance_df))

print("평균 거리: ", distance_df.mean(), "\n최대 거리: ", distance_df.max(), "\n최소 거리: ", distance_df.min())


In [ ]:
#final_df=pd.concat([df,distance_df],axis=1, join='inner')
#df=final_df.dropna(axis=1)
#print(df.head(5))

final_df=pd.concat([df, distance_df, busStop_df, busX, busY], axis=1, join='inner')
df=final_df.dropna(axis=1)
print(df.head(5))

In [ ]:
print(len(df))

df.to_csv("real_final_busStop.csv",encoding="EUC-KR")

In [ ]:
# 동별 plot 그려보기
import seaborn as sns

#sns.boxplot(x="dong", y="distance", data=subset_중앙동)

In [ ]:
#중앙동_distance=subset_중앙동['distance'].mean()

# 그냥 동별 경로당에서 정류장까지 거리를 평균값으로 계산

# 완산구 행정동 거리 평균값
중앙동_mean=(df['distance'].groupby(df["dong"]=="중앙동")).mean()
풍남동_mean=(df['distance'].groupby(df["dong"]=="풍남동")).mean()
노송동_mean=(df['distance'].groupby(df["dong"]=="노송동")).mean()
완산동_mean=(df['distance'].groupby(df["dong"]=="완산동")).mean()
서서학동_mean=(df['distance'].groupby(df["dong"]=="서서학동")).mean()
중화산동_mean=(df['distance'].groupby(df["dong"]=="중화산동")).mean()
평화동_mean=(df['distance'].groupby(df["dong"]=="평화동")).mean()
서신동_mean=(df['distance'].groupby(df["dong"]=="서신동")).mean()
삼천동_mean=(df['distance'].groupby(df["dong"]=="삼천동")).mean()
효자동_mean=(df['distance'].groupby(df["dong"]=="효자동")).mean()

# 덕진구 행정도 거리 평균값
진북동_mean=(df['distance'].groupby(df["dong"]=="진북동")).mean()
인후동_mean=(df['distance'].groupby(df["dong"]=="인후동")).mean()
덕진동_mean=(df['distance'].groupby(df["dong"]=="덕진동")).mean()
금암동_mean=(df['distance'].groupby(df["dong"]=="금암동")).mean()
팔복동_mean=(df['distance'].groupby(df["dong"]=="팔복동")).mean()
우아동_mean=(df['distance'].groupby(df["dong"]=="우아동")).mean()
호성동_mean=(df['distance'].groupby(df["dong"]=="호성동")).mean()
송천동_mean=(df['distance'].groupby(df["dong"]=="송천동")).mean()


In [ ]:
wansan_dict={"중앙동": 중앙동_mean, "풍남동": 풍남동_mean, "노송동": 노송동_mean, "완산동": 완산동_mean, "서서학동": 서서학동_mean,
             "중화산동": 중화산동_mean, "평화동": 평화동_mean, "서신동": 서신동_mean, "삼천동": 삼천동_mean, "효자동": 효자동_mean,}

deokjin_dict={"진북동": 진북동_mean, "인후동": 인후동_mean, "덕진동": 덕진동_mean, "금암동": 금암동_mean, "팔복동": 팔복동_mean, 
              "우아동": 우아동_mean, "호성동": 호성동_mean, "송천동": 송천동_mean,}

wansan=pd.DataFrame(wansan_dict)
deokjin=pd.DataFrame(deokjin_dict)


In [ ]:
wansan=np.transpose(wansan)
deokjin=np.transpose(deokjin)

In [ ]:
dong_distance=pd.concat([wansan,deokjin],axis=0, join='inner') # 완산구 + 덕진구 거리 데이터프레임
dong_distance=dong_distance.dropna(axis=1)
#print(dong_distance.head(5))

dong_distance=dong_distance.drop([False], axis='columns') # False열 삭제


In [ ]:
dong_distance.head(5)